# Урок 2. Введение в подготовку данных для аналитиков. Таблицы фактов и таблицы измерений

1. Скачайте датасет fifаs2.сsv. Проанализируйте его и определите, какие данные являются неполными. Удалите
ненужные колонки и недостающие значения.

2. Найдите в датафрейме полные дубликаты и удалите их. Значения могут быть одинаковыми, но написаны по-разному. Например, может отличаться размер регистра (заглавные и строчные буквы). Особое внимание уделить колонке с названиями команд.

3. Напишите функцию, которая добавит колонку с разбиением возраста по группам: до 20, от 20 до 30, от 30 до
36 и старше 36. Посчитайте количество футболистов в каждой категории.

In [1]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

In [2]:
conf = SparkConf().setAppName('les_2_hw')
sc = SparkContext(conf=conf)

spark = SparkSession.builder.getOrCreate()

csv_file_path = 'data/fifa_s2.csv' 

25/05/02 21:49:43 WARN Utils: Your hostname, ub-xeon-v1 resolves to a loopback address: 127.0.1.1; using 192.168.1.100 instead (on interface enp6s0)
25/05/02 21:49:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/02 21:49:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/02 21:49:45 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [19]:
df = spark.read.csv(csv_file_path, header=True, inferSchema=True)

In [4]:
df.show(3)

+----+---------+---+-----------+-------+---------+-------------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+
|  ID|     Name|Age|Nationality|Overall|Potential|               Club|Value|Wage|Preferred Foot|International Reputation|Skill Moves|Position|Joined|Contract Valid Until|     Height|Weight|Release Clause|
+----+---------+---+-----------+-------+---------+-------------------+-----+----+--------------+------------------------+-----------+--------+------+--------------------+-----------+------+--------------+
|1179|G. Buffon| 40|      Italy|     88|       88|Paris Saint-Germain| 4000|  77|         Right|                       4|          1|      GK|  2018|          01/01/2019|6.333333333| 203.0|        7400.0|
|5479| Casillas| 37|      Spain|     82|       82|           FC Porto| 1500|  10|          Left|                       4|          1|    NULL|  2015|          01/01/2019|6.08333333

In [5]:
df.dtypes

[('ID', 'int'),
 ('Name', 'string'),
 ('Age', 'int'),
 ('Nationality', 'string'),
 ('Overall', 'int'),
 ('Potential', 'int'),
 ('Club', 'string'),
 ('Value', 'int'),
 ('Wage', 'int'),
 ('Preferred Foot', 'string'),
 ('International Reputation', 'int'),
 ('Skill Moves', 'int'),
 ('Position', 'string'),
 ('Joined', 'int'),
 ('Contract Valid Until', 'string'),
 ('Height', 'double'),
 ('Weight', 'double'),
 ('Release Clause', 'double')]

In [6]:
data_col = []
for col in df.columns:
    num_col = df.filter(F.isnull(col)).count()
    if num_col != 0:
        data_col.append(col)
        print(f'{col} - {df.filter(F.isnull(col)).count()} пропусков')

Nationality - 7 пропусков
Club - 241 пропусков
Value - 257 пропусков
International Reputation - 10 пропусков
Skill Moves - 6 пропусков
Position - 21 пропусков
Contract Valid Until - 273 пропусков
Release Clause - 3 пропусков


In [7]:
df_cleans = df

In [8]:
for col in data_col:
    df_cleans = df_cleans.filter(df_cleans[col].isNotNull())

In [9]:
for col in df_cleans.columns:
    print(f'{col} - {df_cleans.filter(F.isnull(col)).count()} пропусков')

ID - 0 пропусков
Name - 0 пропусков
Age - 0 пропусков
Nationality - 0 пропусков
Overall - 0 пропусков
Potential - 0 пропусков
Club - 0 пропусков
Value - 0 пропусков
Wage - 0 пропусков
Preferred Foot - 0 пропусков
International Reputation - 0 пропусков
Skill Moves - 0 пропусков
Position - 0 пропусков
Joined - 0 пропусков
Contract Valid Until - 0 пропусков
Height - 0 пропусков
Weight - 0 пропусков
Release Clause - 0 пропусков


In [10]:
df_cleans.count()

2075

In [11]:
df_cleans = df_cleans.drop('ID')

In [12]:
Age_Group = ['up to 20', 'from 20 to 30', 'from 30 to 36', 'over 36']

In [13]:
def age_group(value: int) -> str:
    if value < 20:
        return Age_Group[0]
    
    elif (value > 20) & (value < 30):
        return Age_Group[1]
    
    elif (value > 30) & (value < 36):
        return Age_Group[2]
    
    else:
        return Age_Group[3]

In [14]:
my_udf = udf(lambda x: age_group(x), StringType())

In [15]:
df_cleans = df_cleans.withColumn('Age_Group', my_udf(df_cleans.Age))

In [20]:
df_cleans.select('Name', 'Age', 'Age_Group').show()

+-----------------+---+-------------+
|             Name|Age|    Age_Group|
+-----------------+---+-------------+
|        G. Buffon| 40|      over 36|
|            Braga| 35|from 30 to 36|
|        A. Robben| 34|from 30 to 36|
|        J. Ibehre| 35|from 30 to 36|
|    Ricardo Costa| 37|      over 36|
|Cristiano Ronaldo| 33|from 30 to 36|
|      D. de Vries| 37|      over 36|
|    S. von Bergen| 35|from 30 to 36|
|     M. Rosenberg| 35|from 30 to 36|
|         S. Proto| 35|from 30 to 36|
|       M. Lehmann| 35|from 30 to 36|
|        J. Murphy| 36|      over 36|
|        C. Gordon| 35|from 30 to 36|
|        A. Collin| 33|from 30 to 36|
|         L. Grant| 35|from 30 to 36|
|       S. Bywater| 37|      over 36|
|         T. Berni| 35|from 30 to 36|
|         R. Green| 38|      over 36|
|        S. Padoin| 34|from 30 to 36|
|  S. Lichtsteiner| 34|from 30 to 36|
+-----------------+---+-------------+
only showing top 20 rows



In [17]:
pivot_df = df_cleans.groupBy().pivot("Age_Group").agg(F.count("Age_Group"))

In [18]:
pivot_df.show()

+-------------+-------------+-------+--------+
|from 20 to 30|from 30 to 36|over 36|up to 20|
+-------------+-------------+-------+--------+
|         1285|          274|    254|     262|
+-------------+-------------+-------+--------+

